# Justin Zarkovacki 2/15/2023
# Transfer Learning MNIST -> KMNIST

# Prepare imports

In [ ]:
import numpy as np
import optuna
import sys
import os

from matplotlib import pyplot as plt

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense
from keras import models, backend as K
from tensorflow.keras import models
from tensorflow.keras.datasets import mnist

print("Done!")

Done!


# Function Definitions and Variables

In [ ]:
epochs = 12
img_rows, img_cols = 28, 28  # Image dimensions

def load(f):
    return np.load(f)['arr_0']
    
def initialize_data(train_im_file, test_im_file, train_lb_file, test_lb_file):
    train_images = load(train_im_file)
    test_images = load(test_im_file)
    train_labels = load(train_lb_file)
    test_labels = load(test_lb_file)
    
    if K.image_data_format() == 'channels_first':
        train_images = train_images.reshape(train_images.shape[0], 1, img_rows, img_cols)
        test_images = test_images.reshape(test_images.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, 1)
        test_images = test_images.reshape(test_images.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    train_images = train_images.astype('float32')
    test_images = test_images.astype('float32')
    train_images /= 255
    test_images /= 255
    print('{} train samples, {} test samples'.format(len(train_images), len(test_images)))
    
    return tuple([train_images, test_images, train_labels, test_labels, input_shape])

# Helper to create the graphics
def create_visuals(graph_title, model_hist, test_images, test_labels):
    accuracy_data = model_hist.history['accuracy']
    val_accuracy_data = model_hist.history['val_accuracy']

    lower_bound = min(min(accuracy_data), min(val_accuracy_data))

    plt.plot(accuracy_data, label='Train Accuracy')
    plt.plot(val_accuracy_data, label = 'Validation Accuracy')

    plt.title(graph_title)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([lower_bound - 0.01, 1])
    plt.legend(loc='lower right')

    print(accuracy_data[-1])
    print(val_accuracy_data[-1])
#     test_loss, test_acc = model_hist.evaluate(test_images, test_labels, verbose=2)

print("Done!")

Done!


## Load Data

In [ ]:
prefix = 'C:\\Users\\LifeH\\SoftwareDevelopment\\KanjiCharacterRecognition\\'
datasets = {
    "km_tr_i" : prefix + "Datasets\\Kuzushiji-MNIST - train-imgs.npz",
    "km_te_i" : prefix + "Datasets\\Kuzushiji-MNIST - test-imgs.npz",
    "km_tr_l" : prefix + "Datasets\\Kuzushiji-MNIST - train-labels.npz",
    "km_te_l" : prefix + "Datasets\\Kuzushiji-MNIST - test-labels.npz"
}

# Load MNIST Data
(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = mnist.load_data()
mnist_input_shape = (28, 28, 1)
mnist_classes = 10
mnist_epochs = 15

# Load KMNIST Data
dataset = initialize_data(datasets["km_tr_i"], datasets["km_te_i"], datasets["km_tr_l"], datasets["km_te_l"])
kmnist_train_images = dataset[0]
kmnist_test_images = dataset[1]
kmnist_train_labels = dataset[2]
kmnist_test_labels  = dataset[3]
kmnist_input_shape = dataset[4]
kmnist_classes = 10
kmnist_epochs = 15

batches = 128
num_trials = 25

print("Done!")

11490434/11490434 [==============================] - 0s 0us/step
60000 train samples, 10000 test samples
Done!


This notebook will create an ensemble model for KMNIST character recognition. It wil be composed of 2 basic models, and one transfer learning model. Knowledge from MNIST will be transfered to KMNIST.

# Creating KMNIST Model 1

In [ ]:
def kmnist_objective1(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 3)
    l1_filters = trial.suggest_int('first_layer_kernel', 32, 54)
    l2_filters = trial.suggest_int('second_layer_kernel', 20, 64)
    l1_activation = trial.suggest_categorical('first_layer_activation', ['relu', 'sigmoid', 'tanh'])
    l2_activation = trial.suggest_categorical('second_layer_activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 80)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    # Design model
    kmnist1_design = Sequential()
    kmnist1_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=kmnist_input_shape))
    kmnist1_design.add(Dropout(dropout))
    kmnist1_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=kmnist_input_shape))
    kmnist1_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    kmnist1_design.add(Flatten())
    kmnist1_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    kmnist1_design.add(Dense(kmnist_classes))

    kmnist1_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(kmnist1_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    kmnist1_history = kmnist1_design.fit(kmnist_train_images, kmnist_train_labels, epochs=kmnist_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(kmnist_test_images, kmnist_test_labels))

    # Important metric for optuna to optimize over
    return kmnist1_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 1
fileName = prefix + 'KanjiModelDevelopmentOutputs\\KMNIST_Model_1.txt'
orig_stdout = sys.stdout
with open(fileName, 'w') as f:
    sys.stdout = f
    kmnist_study1 = optuna.create_study(direction='maximize', study_name="KMNIST-1")
    kmnist_study1.optimize(kmnist_objective1, n_trials=10)
    sys.stdout = orig_stdout
    f.close()

In [ ]:
# Print the info from the best trial
print(f'Best trial info:\n{kmnist_study1.best_trial}\n')
for param, value in kmnist_study1.best_params.items():
    print(f'Param: {param}\tValue: {value}')

Best trial info:
FrozenTrial(number=11, values=[0.9496999979019165], datetime_start=datetime.datetime(2023, 2, 17, 19, 1, 50, 150095), datetime_complete=datetime.datetime(2023, 2, 17, 19, 3, 14, 414231), params={'kernel_size': 3, 'first_layer_kernel': 50, 'second_layer_kernel': 63, 'first_layer_activation': 'relu', 'second_layer_activation': 'relu', 'dropout': 0.1646000975336841, 'average_pooling_size': 4, 'dense_layer_size': 80, 'dense_layer_activation': 'relu'}, distributions={'kernel_size': IntDistribution(high=3, log=False, low=2, step=1), 'first_layer_kernel': IntDistribution(high=54, log=False, low=32, step=1), 'second_layer_kernel': IntDistribution(high=64, log=False, low=20, step=1), 'first_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'second_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'dropout': FloatDistribution(high=0.3, log=False, low=0.15, step=None), 'average_pooling_size': IntDistribution(high=4, lo

In [ ]:
# Optuna doesn't save the best model. You must rebuild it and save it.
kern_size	= 3
l1_filters = 50
l2_filters = 63
l1_activation = "relu"
l2_activation = "relu"
dropout	= 0.1646000975336841
average_pooling_size = 4
dense_layer_size	= 80
dense_layer_activation = "relu"

kmnist1 = Sequential()
kmnist1.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=kmnist_input_shape))
kmnist1.add(Dropout(dropout))
kmnist1.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=kmnist_input_shape))
kmnist1.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
kmnist1.add(Flatten())
kmnist1.add(Dense(dense_layer_size, activation=dense_layer_activation))
kmnist1.add(Dense(kmnist_classes))

kmnist1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

# kmnist1.summary()

In [ ]:
tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

kmnist1_optuna_history = kmnist1.fit(kmnist_train_images, kmnist_train_labels, epochs=kmnist_epochs, batch_size=batches,
                            callbacks=callback, validation_data=(kmnist_test_images, kmnist_test_labels))
kmnist1.save(prefix + 'kmnist_models\\kmnist1.h5', save_format='h5')

Epoch 1/15


/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


469/469 [==============================] - 23s 48ms/step - loss: 0.4464 - sparse_categorical_crossentropy: 1.7126 - accuracy: 0.8651 - val_loss: 0.4669 - val_sparse_categorical_crossentropy: 1.6704 - val_accuracy: 0.8579
Epoch 2/15
469/469 [==============================] - 24s 52ms/step - loss: 0.1558 - sparse_categorical_crossentropy: 1.4541 - accuracy: 0.9534 - val_loss: 0.3582 - val_sparse_categorical_crossentropy: 1.5974 - val_accuracy: 0.8971
Epoch 3/15
469/469 [==============================] - 30s 63ms/step - loss: 0.1043 - sparse_categorical_crossentropy: 1.3610 - accuracy: 0.9693 - val_loss: 0.2885 - val_sparse_categorical_crossentropy: 1.5216 - val_accuracy: 0.9184
Epoch 4/15
469/469 [==============================] - 24s 52ms/step - loss: 0.0798 - sparse_categorical_crossentropy: 1.3201 - accuracy: 0.9761 - val_loss: 0.2722 - val_sparse_categorical_crossentropy: 1.4548 - val_accuracy: 0.9285
Epoch 5/15
469/469 [==============================] - 22s 48ms/step - loss: 0.0615 

# Creating KMNIST Model 2

In [ ]:
def kmnist_objective2(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 3)
    l1_filters = trial.suggest_int('first_layer_kernel', 20, 40)
    l2_filters = trial.suggest_int('second_layer_kernel', 40, 64)
    l1_activation = trial.suggest_categorical('first_layer_activation', ['relu', 'sigmoid', 'tanh'])
    l2_activation = trial.suggest_categorical('second_layer_activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 80)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    # Design model
    kmnist2_design = Sequential()
    kmnist2_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=kmnist_input_shape))
    kmnist2_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=kmnist_input_shape))
    kmnist2_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    kmnist2_design.add(Dropout(dropout))
    kmnist2_design.add(Flatten())
    kmnist2_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    kmnist2_design.add(Dense(kmnist_classes))

    kmnist2_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(kmnist2_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    kmnist2_history = kmnist2_design.fit(kmnist_train_images, kmnist_train_labels, epochs=kmnist_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(kmnist_test_images, kmnist_test_labels))

    # Important metric for optuna to optimize over
    return kmnist2_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 2
fileName = prefix + 'KanjiModelDevelopmentOutputs\\KMNIST_Model_2.txt'
orig_stdout = sys.stdout
with open(fileName, 'w') as f:
    sys.stdout = f
    kmnist_study2 = optuna.create_study(direction='maximize', study_name="KMNIST-2")
    kmnist_study2.optimize(kmnist_objective2, n_trials=10)
    sys.stdout = orig_stdout
    f.close()

In [ ]:
# Print the info from the best trial
print(f'Best trial info:\n{kmnist_study2.best_trial}\n')
for param, value in kmnist_study2.best_params.items():
    print(f'Param: {param}\tValue: {value}')

Best trial info:
FrozenTrial(number=22, values=[0.9574000239372253], datetime_start=datetime.datetime(2023, 2, 17, 22, 20, 7, 249500), datetime_complete=datetime.datetime(2023, 2, 17, 22, 25, 50, 261204), params={'kernel_size': 3, 'first_layer_kernel': 36, 'second_layer_kernel': 51, 'first_layer_activation': 'tanh', 'second_layer_activation': 'relu', 'dropout': 0.23565992578719316, 'average_pooling_size': 4, 'dense_layer_size': 80, 'dense_layer_activation': 'relu'}, distributions={'kernel_size': IntDistribution(high=3, log=False, low=2, step=1), 'first_layer_kernel': IntDistribution(high=40, log=False, low=20, step=1), 'second_layer_kernel': IntDistribution(high=64, log=False, low=40, step=1), 'first_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'second_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'dropout': FloatDistribution(high=0.3, log=False, low=0.15, step=None), 'average_pooling_size': IntDistribution(high=4, 

In [ ]:
# Optuna doesn't save the best model. You must rebuild it and save it.
kern_size	= 3
l1_filters = 36
l2_filters = 51
l1_activation = "tanh"
l2_activation = "relu"
dropout	= 0.23565992578719316
average_pooling_size = 4
dense_layer_size	= 80
dense_layer_activation = "relu"

kmnist2 = Sequential()
kmnist2.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=kmnist_input_shape))
kmnist2.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=kmnist_input_shape))
kmnist2.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
kmnist2.add(Dropout(dropout))
kmnist2.add(Flatten())
kmnist2.add(Dense(dense_layer_size, activation=dense_layer_activation))
kmnist2.add(Dense(kmnist_classes))

kmnist2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

# kmnist2.summary()

In [ ]:
tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

kmnist2_optuna_history = kmnist2.fit(kmnist_train_images, kmnist_train_labels, epochs=kmnist_epochs, batch_size=batches,
                            callbacks=callback, validation_data=(kmnist_test_images, kmnist_test_labels))

kmnist2.save(prefix + 'kmnist_models\\kmnist2.h5', save_format='h5')

Epoch 1/15


/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


469/469 [==============================] - 22s 46ms/step - loss: 0.4426 - sparse_categorical_crossentropy: 1.6114 - accuracy: 0.8669 - val_loss: 0.4276 - val_sparse_categorical_crossentropy: 1.5325 - val_accuracy: 0.8725
Epoch 2/15
469/469 [==============================] - 22s 48ms/step - loss: 0.1699 - sparse_categorical_crossentropy: 1.2471 - accuracy: 0.9488 - val_loss: 0.3394 - val_sparse_categorical_crossentropy: 1.3649 - val_accuracy: 0.8984
Epoch 3/15
469/469 [==============================] - 25s 53ms/step - loss: 0.1225 - sparse_categorical_crossentropy: 1.1881 - accuracy: 0.9626 - val_loss: 0.2780 - val_sparse_categorical_crossentropy: 1.2739 - val_accuracy: 0.9182
Epoch 4/15
469/469 [==============================] - 24s 51ms/step - loss: 0.0977 - sparse_categorical_crossentropy: 1.1112 - accuracy: 0.9708 - val_loss: 0.2547 - val_sparse_categorical_crossentropy: 1.2503 - val_accuracy: 0.9271
Epoch 5/15
469/469 [==============================] - 22s 47ms/step - loss: 0.0784 

# MNIST to KMNIST Transfer Learning

## Create MNIST Base

In [ ]:
def mnist_base_objective(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 3)
    l1_filters = trial.suggest_int('first_layer_kernel', 20, 40)
    l2_filters = trial.suggest_int('second_layer_kernel', 40, 64)
    l1_activation = trial.suggest_categorical('first_layer_activation', ['relu', 'sigmoid', 'tanh'])
    l2_activation = trial.suggest_categorical('second_layer_activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 80)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    # Design model
    mnist_base_design = Sequential()
    mnist_base_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=mnist_input_shape))
    mnist_base_design.add(Dropout(dropout))
    mnist_base_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=mnist_input_shape))
    # mnist_base_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    mnist_base_design.add(Flatten())
    mnist_base_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    mnist_base_design.add(Dense(mnist_classes))

    mnist_base_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(mnist_base_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    mnist_base_history = mnist_base_design.fit(mnist_train_images, mnist_train_labels,
                    epochs=mnist_epochs, batch_size=batches, callbacks=callback,
                    validation_data=(mnist_test_images, mnist_test_labels))

    # Important metric for optuna to optimize over
    return mnist_base_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 3
fileName = prefix + 'KanjiModelDevelopmentOutputs\\MNIST_Base_Model.txt'
orig_stdout = sys.stdout
with open(fileName, 'w') as f:
    sys.stdout = f
    mnist_base_study = optuna.create_study(direction='maximize', study_name="MNIST-Base")
    mnist_base_study.optimize(mnist_base_objective, n_trials=10)
    sys.stdout = orig_stdout
    f.close()

In [ ]:
# Print the info from the best trial
print(f'Best trial info:\n{mnist_base_study.best_trial}\n')
for param, value in mnist_base_study.best_params.items():
    print(f'Param: {param}\tValue: {value}')

Best trial info:
FrozenTrial(number=7, values=[0.9825999736785889], datetime_start=datetime.datetime(2023, 2, 19, 4, 48, 39, 409443), datetime_complete=datetime.datetime(2023, 2, 19, 4, 49, 44, 777393), params={'kernel_size': 2, 'first_layer_kernel': 29, 'second_layer_kernel': 52, 'first_layer_activation': 'relu', 'second_layer_activation': 'tanh', 'dropout': 0.17435848899058518, 'average_pooling_size': 4, 'dense_layer_size': 65, 'dense_layer_activation': 'sigmoid'}, distributions={'kernel_size': IntDistribution(high=3, log=False, low=2, step=1), 'first_layer_kernel': IntDistribution(high=40, log=False, low=20, step=1), 'second_layer_kernel': IntDistribution(high=64, log=False, low=40, step=1), 'first_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'second_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'dropout': FloatDistribution(high=0.3, log=False, low=0.15, step=None), 'average_pooling_size': IntDistribution(high=4,

In [ ]:
# Optuna doesn't save the best model. You must rebuild it and save it.
kern_size	= 2
l1_filters = 29
l2_filters = 52
l1_activation = "relu"
l2_activation = "tanh"
dropout	= 0.17435848899058518
average_pooling_size = 4
dense_layer_size	= 65
dense_layer_activation = "sigmoid"

mnist_base = Sequential()
mnist_base.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=mnist_input_shape))
mnist_base.add(Dropout(dropout))
mnist_base.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=mnist_input_shape))
mnist_base.add(Flatten())
mnist_base.add(Dense(dense_layer_size, activation=dense_layer_activation))
mnist_base.add(Dense(mnist_classes))

mnist_base.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

mnist_optuna_history = mnist_base.fit(mnist_train_images, mnist_train_labels, batch_size=batches, epochs=epochs,
                                callbacks=callback, validation_data=(mnist_test_images, mnist_test_labels))

mnist_base.save(prefix + 'mnist_base.h5', save_format='h5')

Epoch 1/12


/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


469/469 [==============================] - 22s 47ms/step - loss: 0.2552 - sparse_categorical_crossentropy: 0.9303 - accuracy: 0.9429 - val_loss: 0.0936 - val_sparse_categorical_crossentropy: 0.6725 - val_accuracy: 0.9782
Epoch 2/12
469/469 [==============================] - 21s 45ms/step - loss: 0.0775 - sparse_categorical_crossentropy: 0.6272 - accuracy: 0.9817 - val_loss: 0.0687 - val_sparse_categorical_crossentropy: 0.6201 - val_accuracy: 0.9825
Epoch 3/12
469/469 [==============================] - 23s 48ms/step - loss: 0.0539 - sparse_categorical_crossentropy: 0.5720 - accuracy: 0.9865 - val_loss: 0.0645 - val_sparse_categorical_crossentropy: 0.5701 - val_accuracy: 0.9803
Epoch 4/12
469/469 [==============================] - 23s 48ms/step - loss: 0.0424 - sparse_categorical_crossentropy: 0.5619 - accuracy: 0.9893 - val_loss: 0.0633 - val_sparse_categorical_crossentropy: 0.5910 - val_accuracy: 0.9803
Epoch 5/12
469/469 [==============================] - 21s 44ms/step - loss: 0.0360 

## Create KMNIST Top

In [ ]:
import os
import random
from skimage.transform import rescale
from skimage import io

# Loading a base model requires you pop all dimensionality flattening layers
def load_base_model(filename):
    bm = models.load_model(filename)
    bm.pop()
    bm.pop()
    bm.pop()
    bm.trainable = False
    return bm

In [ ]:
def kmnist_top_objective(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 3)
    l1_filters = trial.suggest_int('first_layer_kernel', 32, 54)
    l2_filters = trial.suggest_int('second_layer_kernel', 64, 96)
    l1_activation = trial.suggest_categorical('first_layer_activation', ['relu', 'sigmoid', 'tanh'])
    l2_activation = trial.suggest_categorical('second_layer_activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 128)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    base_model = load_base_model(prefix + 'mnist_base.h5')
    kmnist_in_shape = (28, 28, 1)

    # Design model
    kmnist_top_design = Sequential()
    kmnist_top_design.add(base_model)
    kmnist_top_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=kmnist_in_shape))
    kmnist_top_design.add(Dropout(dropout))
    kmnist_top_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=kmnist_in_shape))
    kmnist_top_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    kmnist_top_design.add(Flatten())
    kmnist_top_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    kmnist_top_design.add(Dense(kmnist_classes))

    kmnist_top_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(kmnist_top_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    kmnist_top_history = kmnist_top_design.fit(kmnist_train_images, kmnist_train_labels, epochs=kmnist_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(kmnist_test_images, kmnist_test_labels))

    # Important metric for optuna to optimize over
    return kmnist_top_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 4
fileName = prefix + 'KanjiModelDevelopmentOutputs\\KMNIST_TL_Model.txt'
orig_stdout = sys.stdout
with open(fileName, 'w') as f:
    sys.stdout = f
    kmnist_TL_study = optuna.create_study(direction='maximize', study_name="KMNIST-TL-Results")
    kmnist_TL_study.optimize(kmnist_top_objective, n_trials=10)
    sys.stdout = orig_stdout
    f.close()

In [ ]:
# Print the info from the best trial
print(f'Best trial info:\n{kmnist_TL_study.best_trial}\n')
for param, value in kmnist_TL_study.best_params.items():
    print(f'Param: {param}\tValue: {value}')

Best trial info:
FrozenTrial(number=24, values=[0.9527999758720398], datetime_start=datetime.datetime(2023, 2, 19, 7, 52, 0, 790831), datetime_complete=datetime.datetime(2023, 2, 19, 7, 58, 23, 288938), params={'kernel_size': 3, 'first_layer_kernel': 39, 'second_layer_kernel': 96, 'first_layer_activation': 'relu', 'second_layer_activation': 'relu', 'dropout': 0.15487543805908655, 'average_pooling_size': 3, 'dense_layer_size': 122, 'dense_layer_activation': 'relu'}, distributions={'kernel_size': IntDistribution(high=3, log=False, low=2, step=1), 'first_layer_kernel': IntDistribution(high=54, log=False, low=32, step=1), 'second_layer_kernel': IntDistribution(high=96, log=False, low=64, step=1), 'first_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'second_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'dropout': FloatDistribution(high=0.3, log=False, low=0.15, step=None), 'average_pooling_size': IntDistribution(high=4, l

In [ ]:
# Optuna doesn't save the best model. You must rebuild it and save it.
base_model = load_base_model(prefix + 'mnist_base.h5')
kern_size	= 3
l1_filters = 39
l2_filters = 96
l1_activation = "relu"
l2_activation = "relu"
dropout	= 0.15487543805908655
average_pooling_size = 3
dense_layer_size	= 122
dense_layer_activation = "relu"

kmnist_TL = Sequential()
kmnist_TL.add(base_model)
kmnist_TL.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=kmnist_input_shape))
kmnist_TL.add(Dropout(dropout))
kmnist_TL.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=kmnist_input_shape))
kmnist_TL.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
kmnist_TL.add(Flatten())
kmnist_TL.add(Dense(dense_layer_size, activation=dense_layer_activation))
kmnist_TL.add(Dense(kmnist_classes))

kmnist_TL.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

kmnist_TL_optuna_history = kmnist_TL.fit(kmnist_train_images, kmnist_train_labels, epochs=kmnist_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(kmnist_test_images, kmnist_test_labels))

kmnist_TL.save(prefix + 'kmnist_models\\kmnist_TL.h5', save_format='h5')

/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/15
469/469 [==============================] - 38s 65ms/step - loss: 0.3917 - sparse_categorical_crossentropy: 1.6688 - accuracy: 0.8815 - val_loss: 0.3897 - val_sparse_categorical_crossentropy: 1.6073 - val_accuracy: 0.8843
Epoch 2/15
469/469 [==============================] - 26s 55ms/step - loss: 0.1194 - sparse_categorical_crossentropy: 1.3551 - accuracy: 0.9650 - val_loss: 0.2729 - val_sparse_categorical_crossentropy: 1.4412 - val_accuracy: 0.9226
Epoch 3/15
469/469 [==============================] - 26s 56ms/step - loss: 0.0769 - sparse_categorical_crossentropy: 1.1759 - accuracy: 0.9762 - val_loss: 0.2380 - val_sparse_categorical_crossentropy: 1.2103 - val_accuracy: 0.9358
Epoch 4/15
469/469 [==============================] - 26s 56ms/step - loss: 0.0566 - sparse_categorical_crossentropy: 1.0691 - accuracy: 0.9822 - val_loss: 0.2006 - val_sparse_categorical_crossentropy: 1.1933 - val_accuracy: 0.9452
Epoch 5/15
469/469 [==============================] - 27s 57ms/step - lo